In [ ]:
# import json

# with open('bot_tmp.log', encoding='utf-8') as f:
#     lines = [line.split(" --- ") for line in f.readlines()]

#     res = {'files': []}

#     for idx, line in enumerate(lines):
#         entry = {
#             'id': idx + 1,
#             'type': line[1].strip()[1:-1],
#             'name': line[0].strip()[1:-1],
#             'uploaded_date': line[2].strip()[1:-1]
#         }

#         res['files'].append(entry)

#     with open('files_info.json', 'w', encoding='utf-8') as f:
#         json.dump(res, f, indent=4, ensure_ascii=False)

In [ ]:
# WARNING -> This doesn't work reliable.... It deletes data that are matched!!!!
# import json

# auxiliaryList = []

# with open('../files_info.json', 'r+', encoding='utf-8') as f:
#     data = json.load(f)

#     for info in data["files"]:
#         info["id"] = 1

#     for idx, info in enumerate(data["files"]):
#         if info not in auxiliaryList:
#             auxiliaryList.append(info)

#     for idx, info in enumerate(auxiliaryList):
#         info["id"] = idx + 1

#     print(len(data['files']))
    
# with open('../files_info.json', 'w', encoding='utf-8') as f:
#     data["files"] = auxiliaryList
#     json.dump(data, f, indent=4, ensure_ascii=False)

In [3]:
import os
import time
import glob
import json
import string
import platform
import unicodedata
from selenium import webdriver
from json.decoder import JSONDecodeError
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Packages & Libraries
import json
from json.decoder import JSONDecodeError
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

from constants import (
    secrets,
    encoding,
    target_file_type,
    log_file_location,
    normalization_form,
    explicit_wait_time,
    tracker_file_location,
)

from helperFunctions import (
    login,
    updateLog,
    waitNSeconds,
    downloadFile,
    loadMoreFiles,
    appendFilesInfo,
    initializeWebpage,
    checkDownloadStatus,
    initializeWebDriver,
    waitToFinishDownload,
    getExistingFilesInfo,
)

driver, wait = initializeWebDriver()
login(driver, wait)

In [4]:
initializeWebpage(driver, "https://www.facebook.com/groups/201623576939858/files/")
"""
 targets identifiers -> this identifiers will change continuously so update it according to your needs 

"""
download_button_cssSelector = "a[href*='https://www.facebook.com/download/']"
permalink_xpath = "//a[contains(@href, 'https://www.facebook.com/groups/201623576939858/permalink/')]"  # by default finds 15 per-scroll
fileOption_xpath = "//div[@aria-label='File options']"  # by default finds 15 per-scroll

# Target identifiers
# They change too often so they have to be present here
fileName_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v lrazzd5p oo9gr5id hzawbc8m']"  # find 15 per-scroll
fileTypeDate_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn iv3no6db e9vueds3 j5wam9gi b1v8xokw oo9gr5id hzawbc8m']"  # find 30 per-scroll

In [ ]:
# def compareString(s1, s2):
#     # string will be normalized before coming here
#     # s1 = unicodedata.normalize(normalization_form, s1)
#     # s2 = unicodedata.normalize(normalization_form, s2)

#     # Removing whitespace in the string before comparing
#     # because when file saves in the machine it seems to add whitepsaces after '-'
#     # remove = string.punctuation + string.whitespace
#     remove = string.whitespace
#     mapping = {ord(c): None for c in remove}

#     return s1.translate(mapping) == s2.translate(mapping)


# def getExistingFilesInfo():
#     with open(tracker_file_location, "r", encoding=encoding) as f:
#         try:
#             files_info = json.load(f)["files"]
#         except JSONDecodeError:
#             files_info = []

#     tracked_files = sorted(
#         [
#             [
#                 unicodedata.normalize(normalization_form, info["uploaded_date"]),
#                 unicodedata.normalize(normalization_form, info["name"]),
#             ]
#             for info in files_info
#         ]
#     )

#     return [], tracked_files


# def binarySearch(item, itemList, multipleCheck=False):
#     left = 0
#     right = len(itemList) - 1

#     while left <= right:
#         mid = left + (right - left) // 2

#         if multipleCheck:
#             if compareString(itemList[mid][0], item[0]) and compareString(
#                 itemList[mid][1], item[1]
#             ):
#                 return mid
#             elif itemList[mid][0] > item[0]:
#                 right = mid - 1
#             else:
#                 left = mid + 1
#         else:
#             if compareString(itemList[mid], item):
#                 return mid
#             elif itemList[mid] > item:
#                 right = mid - 1
#             else:
#                 left = mid + 1

#     return -1


In [13]:
import time

with open(tracker_file_location, "r", encoding=encoding) as f:
    try:
        registered_files = json.load(f)["files"]
    except JSONDecodeError:
        registered_files = []

def compareString(s1, s2):
    s1 = unicodedata.normalize(normalization_form, s1)
    s2 = unicodedata.normalize(normalization_form, s2)

    remove = string.whitespace
    mapping = {ord(c): None for c in remove}

    return s1.translate(mapping) == s2.translate(mapping)


def searchFile(match_item, itemList, multipleCheck=False):
    if multipleCheck:
        for idx, item in enumerate(itemList):
            if compareString(match_item["name"], item["name"]) and compareString(
                match_item["uploaded_date"], item["uploaded_date"]
            ):
                return idx

    else:
        for idx, item in enumerate(itemList):
            if compareString(item, match_item):
                return idx

    return -1


def loadMoreFiles(driver, files_to_load, identifier, timeout=600, n_scroll=1):
    """
    Scroll down to load more files
    wait 'timeout=10 minutes' before determining that there is no more files to load

    """
    # Do this operation(scroll to load) for n times
    for _ in range(n_scroll):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time_elapsed = 0
        current_len = len(files_to_load[0])

        """
        If the first one is loaded then all others will surely get loaded
        so you don't have to check the whole length of 'files_to_load' array
        but if you want you can do something like this ->
        current_len = sum(len(i) for i in files_to_load)
        
        """
        while time_elapsed < timeout:
            # Computation here gets much more expensive as more and more data loads
            # so it is not viable to do this check every second
            start_time = time.time()
            waitNSeconds(5)

            first_data = driver.find_elements(By.XPATH, identifier[0])

            # Iterate through all the files that are needed to be loaded
            if current_len < len(first_data):
                files_to_load[0].extend(
                    [
                        element
                        for element in first_data
                        if element not in files_to_load[0]
                    ]
                )

                for idx in range(1, len(files_to_load)):
                    files_to_load[idx].extend(
                        [
                            element
                            for element in driver.find_elements(By.XPATH, identifier[idx])
                            if element not in files_to_load[idx]
                        ]
                    )

                break

            time_elapsed += time.time() - start_time

    return files_to_load


In [14]:
j, k = 0, 1

# Refresh existing files info every time this cell loads
# downloaded_files, tracked_files = getExistingFilesInfo()

# Grabbing initial loadeded target
try:
    file_option_buttons = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileOption_xpath))
    )
    file_names = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileName_xpath))
    )
    file_types_and_dates = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileTypeDate_xpath))
    )

    # This is the unique identifier of a post/file
    post_permalink = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, permalink_xpath))
    )
except TimeoutException:
    print("XPATH's identifier (class names) has changed again! 🤯")


""" MAIN LOOP """
for idx, button in enumerate(file_option_buttons):
    try:
        _name = file_names[idx].text
        _type = file_types_and_dates[j].text
        _date = file_types_and_dates[k].text
        _permalink = post_permalink[idx].get_attribute("href")

        # as 'file_types_and_dates' contains both file-type and date
        j += 2
        k += 2

        # Scrolling after it reaches at the end of the list to load more files
        if button is file_option_buttons[-1]:
            [
                file_option_buttons,
                file_names,
                file_types_and_dates,
                post_permalink,
            ] = loadMoreFiles(
                driver,
                [file_option_buttons, file_names, file_types_and_dates, post_permalink],
                [fileOption_xpath, fileName_xpath, fileTypeDate_xpath, permalink_xpath],
            )

            updateLog("\nTotal Loaded Files: {}\n".format(len(file_option_buttons)))

        """
        Download pdf files only
        Cannot check this before the scroll because
        here is a possibility to have more than one page worth of non-pdf files
        
        """
        if not _type == target_file_type.upper():
            updateLog(
                '\n😪 Skipping ({}): "{} --- {}", 🤔 Reason: FILE_TYPE: "{}"'.format(
                    idx + 1, _name, _date, _type
                )
            )
            continue

        isExists = searchFile(
            {"name": _name, "uploaded_date": _date}, registered_files, True
        )

        if isExists != -1:
            # waitNSeconds(0.5)

            with open(tracker_file_location, "r+", encoding="utf-8") as f:
                data = json.load(f)

                for tidx, info in enumerate(data["files"]):
                    if compareString(info["name"], _name) and compareString(
                        info["uploaded_date"], _date
                    ):
                        if "post_permalink" in info:
                            updateLog(
                                "\nSkipped -> post_permalink already exists: {} -- {}".format(
                                    _name, _date
                                )
                            )
                            break

                        data["files"][tidx] = {
                            "id": info["id"],
                            "type": info["type"],
                            "name": info["name"],
                            "uploaded_date": info["uploaded_date"],
                            "post_permalink": _permalink,
                        }

                        updateLog(
                            "\n*** Info updated: {} -- {} -- {}".format(
                                info["id"], _name, _date
                            )
                        )

                        f.seek(0)
                        json.dump(data, f, indent=4, ensure_ascii=False)

                        data.clear()

                        # isExists = -2
                        break

            registered_files.pop(isExists)

            # if isExists == -2:
            #     break

            if not registered_files:
                updateLog("\n\n\n\nALLLLLLLLLL DONE\n\n\n")
                break

        else:
            updateLog("\nSkipped: {} -- {}".format(_name, _date))
        print("Progress:", idx + 1)

    except Exception as e:
        updateLog("\n*** ERROR at {}, date: {} ***\n--> {}\n".format(_name, _date, e))



Skipped: পক্ষীপ্রেমী রাজা রাজেন্দ্রলাল মল্লিক – চন্দন মুখার্জ্জী.pdf -- March 17, 2022 at 6:40 PM
Progress: 1

Skipped: আণবিক যুগের পথিকৃৎ মাদাম কুরী – সত্যেন্দ্রনাথ বসু.pdf -- March 17, 2022 at 6:37 PM
Progress: 2

Skipped: কয়েক ধরনের সংখ্যা – বিশ্বনাথ বসু.pdf -- March 17, 2022 at 5:37 PM
Progress: 3

Skipped: এদেশের প্রথম আধুনিক চিকিৎসাবিজ্ঞান গবেষক সুর্যকুমার গুডিব চক্রবর্তী – রণতোষ চক্রবর্তী.pdf -- March 17, 2022 at 5:24 PM
Progress: 4

Skipped: অশঙ্কিনী কাদম্বিনী নির্বারিত নির্ঝর – প্রবীর মুখোপাধ্যায়.pdf -- March 17, 2022 at 5:13 PM
Progress: 5

Skipped: প্রথম বাঙালী মহিলা চিকিৎসক গবেষক সুজাতা চৌধুরী – জয়শ্রী দত্ত.pdf -- March 17, 2022 at 4:59 PM
Progress: 6

Skipped: রম্যানি বীক্ষ্য (কামরূপ পর্ব) - সুবোধ কুমার চক্রবর্তী.pdf -- March 17, 2022 at 3:53 PM
Progress: 7

Skipped: লোকশিক্ষা ও কবিতা.pdf -- March 17, 2022 at 3:38 PM
Progress: 8

Skipped: সার্কাস - শৈলেন ঘোষ.pdf -- March 17, 2022 at 3:10 PM
Progress: 9

Skipped: মায়ের সোনা ছেলে আফজল - শৈলেন ঘোষ.pdf -- March 17, 2022 at 3:

In [ ]:
# with open(tracker_file_location, "r+", encoding="utf-8") as f:
#     data = json.load(f)

#     count = 0

#     chk_len = len(data["files"][1])

#     for idx, info in enumerate(data["files"]):
#         if len(info) != chk_len:
#             print(info["id"])
#             # count += 1
    
#     # print(count)

In [23]:
# with open(tracker_file_location, "r+", encoding="utf-8") as f:
#     data = json.load(f)["files"]
#     # size = len(data) - 1

#     name = []
#     nameSet = set()

#     for i in data:
#         name.append(i["name"] + i["uploaded_date"])
#         nameSet.add(i["name"] + i["uploaded_date"])
    
#     name = sorted(name)

#     size1 = len(name)
#     size2 = len(nameSet)

#     print(size1, "SED")

#     # array = []

#     # for i in range(0, size):
#     #     tmp = set()

#     #     for j in range(1, size):
#     #         if compareString(data[i]["name"], data[j]["name"]):
#     #             tmp.add(data[i]["id"])
#     #             tmp.add(data[j]["id"])

#     #     array.append(tmp)

#     # print("**", len(array), "**")

#     # for a in array:
#     #     print(a)
        

5733 SED
